# Tarea 1 Pregunta 1

En esta tarea definiremos una funcion para romper el esquema criptográfico definido en el enunciado. Esta funcion, break_rp, que podemos ver abajo, recibe 3 valores. cipher es un texto cifrado con el esquema. freq es una lista de frecuencias de las letras del lenguaje del mensaje original. distance es una funcion que calcula la distancia entre dos caracteres.

Como no conocemos el largo de la clave, iteraremos linealmente de 1 a el tamaño del mensaje dividido en 50, como especifica el enunciado. En cada iteracion, dependiendo del tamaño de la clave n=len(k) creamos n grupos. dentro de cada unos de estos grupos agregamos las letras del mensaje que fueron codificadas con el caracter i de la llave k. Por ejemplo si tenemos una clave de tamaño 4 y un mensaje:

![](image1.png)

Grupo0: [E, _, S, _, A, E, _, I, A]

Grupo1: [S, M, A, E, _, N, C, F, D]

Grupo2: [T, E, J, S, S, D, O, I, O]

Grupo3: [E, N, E, T, I, O, D, C]

Cada uno de estos grupos deberia tener una frecuencia de letras similar al lenguaje en el que el mensaje esta escrito. Por ejemplo si la letra S tiene una frecuencia de 12.7% lo mas probable es que corresponda a la letra E en el lenguaje ingles. Podemos deducir de esto que E + K_I = S, si codifico E con la clave K entonces recibo S. Asi hemos adivinado el valor de la clave para ese grupo. Repetimos este proceso para todos los grupos y creamos una clave final.

Gracias a nuestra funcion de distancia podemos estimar si la clave que conseguimos es apropiada al comparar la frecuencia del mensaje decifrado con la frencuencia que nos entregaron. Si la distancia es pequeña entonces es probable que la llave sea la correcta y hallamos logrado decifrar el texto.

In [117]:
def break_rp(cipher: str, freq: dict[str, float], distance: callable) -> str:
    # Creamos un diccionario con todas los caracteres del lenguaje entregado
    # y reducimos las frecuencias a 0. Podemos copiar este en el futuro para
    # definir nuevos diccionarios de frecuencia vacios.
    lang = get_alphabet(freq)
    empty_freq_dict = freq.copy()
    for k in empty_freq_dict.keys():
        empty_freq_dict[k] = 0
    most_freq_letter = max(freq, key= lambda i: freq[i])

    # El largo de la clave es desconocido. Iteraremos desde largo 1 en adelante
    key_length = 1
    best_key = []
    best_distance = -1
    
    while key_length <= len(cipher)//50:
        # Creamos una lista de frecuencias y una lista de caracteres para cada
        # uno de los caracteres de la llave. Despues compararemos la distacia
        # para encontrar el caracter en la llave correspondiente
        N = len(freq)
        key = ["" for i in range(key_length)]
        list_of_freqs = [empty_freq_dict.copy() for i in range(key_length)]
        character_strings = [list() for i in range(key_length)]
        for i in range(len(cipher)):
            list_of_freqs[i%key_length][cipher[i]] += 1
            character_strings[i%key_length].append(cipher[i])
        # Una vez creados los grupos, elegimos la letra mas frecuente del grupo y del lenguaje 
        # Asumimos que la diferencia entre esas letras es el valor de la clave.
        for i in range(key_length):
            most_freq_in_group = max(list_of_freqs[i], key= lambda k: list_of_freqs[i][k])
            key[i] = lang[mod(lang.index(most_freq_in_group) - lang.index(most_freq_letter), N)]
        
        # Una vez creada una clave, deciframos el mensaje y evaluamos la frecuencia de las letras
        # en ese mensaje con la frecuencia entregada. Si es pequeña, entonces elegimos una buena clave
        # si no, seguimos iterando con una llave de mayor largo.
        message = dec(cipher, "".join(key), lang)
        dist = distance(message, freq)
        if best_distance < 0 or dist < best_distance:
            best_distance = dist
            best_key = key
        key_length += 1
    return dec(cipher, "".join(best_key), lang)

La funcion de distancia que utilizaremos es similar a la vista en clases. Además definiremos varias funciones auxiliares.

In [118]:
def abs_distance ( string : str , frequencies : dict[str, float]) -> float :
    """
    Arguments :
    string : An abritrary string
    frequencies : A dictionary representing a character frequency
    Returns :
    distance : How distant is the string from the character frequency
    """
    alphabet = get_alphabet(frequencies)
    string_freq = dict.fromkeys(alphabet, 0)
    for c in string:
        string_freq[c] += 1
    
    distance = 0
    for l in alphabet:
        distance += abs(frequencies[l] - string_freq[l])

    return distance

In [119]:
def enc(m: str, k: str, alphabet: list[str]):
    c = ""
    key_length = len(k)
    for i in range(len(m)):
        m_pos = alphabet.index(m[i])
        k_pos = alphabet.index(k[mod(i, key_length)])
        c_pos = mod(m_pos + k_pos, len(alphabet))
        c += alphabet[c_pos]
    return c


def dec(c: str, k: str, alphabet: list[str]):
    m = ""
    key_length = len(k)
    for i in range(len(c)):
        c_pos = alphabet.index(c[i])
        k_pos = alphabet.index(k[i % key_length])
        m_pos = mod(c_pos - k_pos, len(alphabet))
        m += alphabet[m_pos]
    return m


def get_alphabet(freq: dict[(str, float)]):
    return [k for k in freq.keys()]


def mod(a, n):
    ret = a
    while ret >= n:
        ret -= n
    while ret < 0:
        ret += n
    return ret



Veamos un ejemplo. Tomando la frecuencia de letras en el lenguaje ingles, sin simbolos, solo mayusculas.

In [120]:
letterFrequency = {
    'E' : 12.0,
    'T' : 9.10,
    'A' : 8.12,
    'O' : 7.68,
    'I' : 7.31,
    'N' : 6.95,
    'S' : 6.28,
    'R' : 6.02,
    'H' : 5.92,
    'D' : 4.32,
    'L' : 3.98,
    'U' : 2.88,
    'C' : 2.71,
    'M' : 2.61,
    'F' : 2.30,
    'Y' : 2.11,
    'W' : 2.09,
    'G' : 2.03,
    'P' : 1.82,
    'B' : 1.49,
    'V' : 1.11,
    'K' : 0.69,
    'X' : 0.17,
    'Q' : 0.11,
    'J' : 0.10,
    'Z' : 0.07 
}

m = "HELLOTHISISAMESSAGEINALLCAPSWEDONTHAVEANYOTHERMORECHARACTERSOTHERTHANCAPSLETTERSWEHAVETOKEEPWRITINGINENGLISHFORTHISFREQUENCYDISTRIBUTIONTOGUESSTHEKEYCORRECTLYTHISISANENGLISHWORDTHEDISTANCE"
key = "ABC"
c = enc(m, key, get_alphabet(letterFrequency))

print(break_rp(c, letterFrequency, abs_distance))

HELLOTHISISAMESSAGEINALLCAPSWEDONTHAVEANYOTHERMORECHARACTERSOTHERTHANCAPSLETTERSWEHAVETOKEEPWRITINGINENGLISHFORTHISFREQUENCYDISTRIBUTIONTOGUESSTHEKEYCORRECTLYTHISISANENGLISHWORDTHEDISTANCE
